In [ ]:
%pip install sdv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached sdv-1.0.1-py2.py3-none-any.whl (115 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 3.7 MB/s eta 0:00:00
  Using cached boto3-1.26.119-py3-none-any.whl (135 kB)
  Using cached deepecho-0.4.0-py2.py3-none-any.whl (28 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.6 MB/s eta 0:00:00
  Using cached copulas-0.8.0-py2.py3-none-any.whl (53 kB)
  Using cached sdmetrics-0.9.3-py2.py3-none-any.whl (142 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1

In [ ]:
# Run this code if not mounted

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from sdv.single_table import TVAESynthesizer
from sdv.metadata import SingleTableMetadata
import pandas as pd
import numpy as np


## Helper functions

In [ ]:


def create_nir_with_response(df):
    """
    Function that selects columns that have complete spectra, and adds Dry Matter 
    as target. It then returns the dataframe with rows permuted.
    """
    nir = df.loc[:, '750':'1032'].copy()
    nir["DM"] = df["DM"].copy()
    return nir.reindex(np.random.RandomState(seed=1).permutation(nir.index))



### Loading data

In [ ]:
mango_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Master/Mango_nir_vis.csv")
mango_data

,Set,Season,Region,Date,Type,Cultivar,Pop,Temp,DM,285,...,1173,1176,1179,1182,1185,1188,1191,1194,1197,1200
0,Cal,1,NT,2/10/2015,Hard Green,Caly,2,Mid,16.792506,0,...,0,0,0,0,0,0,0,0,0,0
1,Cal,1,NT,2/10/2015,Hard Green,Caly,2,Mid,16.792506,0,...,0,0,0,0,0,0,0,0,0,0
2,Cal,1,NT,2/10/2015,Hard Green,Caly,2,Mid,16.070979,0,...,0,0,0,0,0,0,0,0,0,0
3,Cal,1,NT,2/10/2015,Hard Green,Caly,2,Mid,16.070979,0,...,0,0,0,0,0,0,0,0,0,0
4,Cal,1,NT,2/10/2015,Hard Green,Caly,2,Mid,16.394013,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11686,Val Ext,4,QLD,25/01/2019,Ripen,HG,112,No,18.986948,0,...,0,0,0,0,0,0,0,0,0,0
11687,Val Ext,4,QLD,25/01/2019,Ripen,HG,112,No,19.711791,0,...,0,0,0,0,0,0,0,0,0,0
11688,Val Ext,4,QLD,25/01/2019,Ripen,HG,112,No,19.711791,0,...,0,0,0,0,0,0,0,0,0,0
11689,Val Ext,4,QLD,25/01/2019,Ripen,HG,112,No,19.478639,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Extracting the NIR spectra
nir = create_nir_with_response(mango_data)

### Loading the metadata

In [ ]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=nir)


### Fitting TVAE synthesizer on NIR data

In [ ]:
tvae_synthesizer = TVAESynthesizer(metadata, epochs=50, batch_size=50)
tvae_synthesizer.fit(nir)

ContextualVersionConflict: ignored

Sampling then saving the dataset in a csv-format

In [ ]:
tvae_nir = tvae_synthesizer.sample(len(nir))
tvae_nir.to_csv("/content/drive/MyDrive/Colab Notebooks/Master/tvae_nir.csv", index=False)